In [1]:
from model import Posicao, Extrato
from views import FundoInvestimento, Acao, FundoImobiliario
#from controllers import MainController
import numpy as np
import pandas as pd 
from datetime import datetime

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
posicao_model = Posicao() 
posicao_model.load_data()
extrato = Extrato(2010, 2020)
fundo_investimento = FundoInvestimento(
    posicao=posicao_model.fis, 
    extrato=extrato.extrato_fis
)
acoes = Acao(posicao=posicao_model.acoes, extrato=extrato.extrato_acoes)
fiis = FundoImobiliario(posicao=posicao_model.fiis, extrato=extrato.extrato_fiis)

/Users/gabriellopes/Documents/Pessoal/dev/finance-app/app/classes/model.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.__aportes_fi_hist['Nome'] = self.__aportes_fi_hist['Descricao'].apply(self.__map_fi)
/Users/gabriellopes/Documents/Pessoal/dev/finance-app/app/classes/model.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ir_fi_hist['Nome'] = self.ir_fi_hist['Descricao'].apply(self.__map_fi)


**Mapeado todos os FIs**

Verifica se existe FI no extrato não mapeado

In [3]:
#fundo_investimento.extrato[fundo_investimento.extrato['Nome'].str.contains('unknown')].head()
#extrato.aportes_fi_hist[extrato.aportes_fi_hist['Nome'] == 'unknown']

## Analise FIs

In [4]:
resumo_ = fundo_investimento.resumo

resumo_.head()

,Nome,Total Bruto,mes,ano,data_posicao,index,aporte,retirada,rendimento_resgatado,rendimento,periodo_cont
0,AZ Quest Multi FIC FIM,0.00,1,2010,2010-01-31,0.00,0.00,0.00,0.00,0.00,0.00
1,AZ Quest Multi FIC FIM,0.00,2,2010,2010-02-28,1.00,0.00,0.00,0.00,0.00,0.00
2,AZ Quest Multi FIC FIM,0.00,3,2010,2010-03-31,2.00,0.00,0.00,0.00,0.00,0.00
3,AZ Quest Multi FIC FIM,0.00,4,2010,2010-04-30,3.00,0.00,0.00,0.00,0.00,0.00
4,AZ Quest Multi FIC FIM,0.00,5,2010,2010-05-31,4.00,0.00,0.00,0.00,0.00,0.00


In [5]:
# um fundo especifico

#display(resumo_[resumo_['Nome'] == 'Azul Quantitativo FIM'])

In [6]:
tmp = resumo_.groupby(['Nome'])\
        .agg(total_aporte=('aporte', 'sum'), 
             total_rendimento=('rendimento', 'sum'), 
             total_retirada=('retirada', 'sum')).reset_index()

tmp

,Nome,total_aporte,total_rendimento,total_retirada
0,AZ Quest Multi FIC FIM,"15,000.00","3,591.84","18,591.84"
1,Azul Quantitativo FIM,"15,200.00","2,624.47","17,824.47"
2,BNP Paribas Inflação FI RF,"10,300.00",-131.86,"10,168.14"
3,Bahia AM Maraú FIC de FIM,"73,000.00","12,897.97",0.00
4,Equitas Selection FIC FIA,"19,900.00","-1,156.14",0.00
5,Icatu Vanguarda Pré-Fixado FIRF,"31,700.00",-934.23,"30,765.77"
6,Mauá Macro FIC FIM,"23,500.00","5,382.28",0.00
7,Mirae Asset Multimercado,"16,000.00","3,122.71","19,122.71"
8,NP Hedge FIC FIM,"5,000.00",144.33,"5,144.33"
9,Polo Norte I FIC FIM,"30,000.00","3,556.88",0.00


In [7]:
display(resumo_[resumo_['Nome'] == 'BNP Paribas Inflação FI RF'])

,Nome,Total Bruto,mes,ano,data_posicao,index,aporte,retirada,rendimento_resgatado,rendimento,periodo_cont
0,BNP Paribas Inflação FI RF,0.00,1,2010,2010-01-31,264.00,0.00,0.00,0.00,0.00,0.00
1,BNP Paribas Inflação FI RF,0.00,2,2010,2010-02-28,265.00,0.00,0.00,0.00,0.00,0.00
2,BNP Paribas Inflação FI RF,0.00,3,2010,2010-03-31,266.00,0.00,0.00,0.00,0.00,0.00
3,BNP Paribas Inflação FI RF,0.00,4,2010,2010-04-30,267.00,0.00,0.00,0.00,0.00,0.00
4,BNP Paribas Inflação FI RF,0.00,5,2010,2010-05-31,268.00,0.00,0.00,0.00,0.00,0.00
5,BNP Paribas Inflação FI RF,0.00,6,2010,2010-06-30,269.00,0.00,0.00,0.00,0.00,0.00
6,BNP Paribas Inflação FI RF,0.00,7,2010,2010-07-31,270.00,0.00,0.00,0.00,0.00,0.00
7,BNP Paribas Inflação FI RF,0.00,8,2010,2010-08-31,271.00,0.00,0.00,0.00,0.00,0.00
8,BNP Paribas Inflação FI RF,0.00,9,2010,2010-09-30,272.00,0.00,0.00,0.00,0.00,0.00
9,BNP Paribas Inflação FI RF,0.00,10,2010,2010-10-31,273.00,0.00,0.00,0.00,0.00,0.00


## Analise Ações

In [8]:
df_extacoes = acoes.extrato
df_posacoes = acoes.posicao
df_extfiis = fiis.extrato
df_posfiis = fiis.posicao

In [9]:
df_extacoes['ano'] = df_extacoes['Data'].dt.year
df_extacoes['mes'] = df_extacoes['Data'].dt.month

In [11]:
tmp = acoes.calcula_resumo(2010, 2020).fillna(0)

tmp.groupby('Papel').agg(total_aporte=('aporte', 'sum'), 
                         
                        total_retirada=('retirada', 'sum'))

,total_aporte,total_retirada
Papel,,
AMZO34,"5,011.20",0.00
BPAC11,0.00,0.00
BPAC12,0.00,0.00
BPAC9,0.00,0.00
CASH3,"3,030.00",0.00
COGN3,"7,044.00",0.00
EGIE3,"9,068.00",0.00
ITSA4,"22,505.00",0.00
JHSF3,"12,944.00","7,300.00"


In [12]:
tmp.head()

,Papel,Data_x,ano,mes,aporte,retirada,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period,data_posicao,index,Data_y
0,AMZO34,2020-12-04 00:00:00,"2,020.00",12.00,"5,011.20",0.00,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0.00,0
1,CASH3,2020-12-03 00:00:00,"2,020.00",12.00,"3,030.00",0.00,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0.00,0
2,COGN3,2020-09-15 00:00:00,"2,020.00",9.00,"7,044.00",0.00,1200,0,0,0,0,0,0,1200,5.18,"6,216.00",2020/9,2020-09-30 00:00:00,128.00,2020-09-30 00:00:00
3,EGIE3,2020-08-04 00:00:00,"2,020.00",8.00,"9,068.00",0.00,200,0,0,0,0,0,0,200,42.63,"8,526.00",2020/8,2020-08-31 00:00:00,127.00,2020-08-31 00:00:00
4,ITSA4,2020-05-28 00:00:00,"2,020.00",5.00,"4,550.00",0.00,0,500,0,0,0,0,0,500,8.84,"4,420.00",2020/5,2020-05-29 00:00:00,124.00,2020-05-31 00:00:00


In [ ]:
tmp = df_extacoes.pivot_table(index=['Papel', 'ano', 'mes', 'Data'], columns=['Tipo'], aggfunc=np.sum, fill_value=0).reset_index()
tmp['total_compra'] = tmp.Preco.COMPRA * tmp.Qtde.COMPRA
tmp['total_venda'] = tmp.Preco.VENDA * tmp.Qtde.VENDA

tmp = tmp[['Papel', 'Data', 'ano', 'mes', 'total_compra', 'total_venda']].reset_index().rename(columns=['Papel', 'Data', 'ano', 'mes', 'total_compra', 'total_venda'])
tmp.info()


In [ ]:
display(df_extacoes.sort_values('Papel'))
display(df_posacoes.sort_values(['Papel', 'data_posicao']))

In [ ]:
acoes.calcula_resumo(2010, 2020)

In [ ]:
display(df_extfiis.head())
display(df_posfiis.head())

In [ ]:
acoes.posicao['Papel'].unique()

In [ ]:
fiis.posicao['Papel'].unique()

In [ ]:
acoes.posicao[acoes.posicao['Papel'] == 'MGLU3'].sort_values('data_posicao')

In [ ]:
acoes.posicao.sort_values(['Papel', 'ano', 'mes'])